<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Policzenie-pierwszych-similarity/colabs/Liczenie_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Przygotowanie notatnika

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1128, done.
remote: Counting objects: 100% (626/626), done.
remote: Compressing objects: 100% (349/349), done.
remote: Total 1128 (delta 388), reused 415 (delta 274), pack-reused 502
Receiving objects: 100% (1128/1128), 5.87 MiB | 25.04 MiB/s, done.
Resolving deltas: 100% (615/615), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


## Załadowanie danych z pliku o rozszerzeniu .parquet

In [3]:
from lib.const import JOINED_DATA, DATA_WITH_AWARDS

W poradniku "Tworzenie skrotu do GDrive" w folderze tutorials jest poradnik jak sprawić żeby poniższa lokalizacja się poprawnie ładowała.


In [4]:
data = pd.read_parquet(JOINED_DATA)
# TODO przepiąć się na nowszy plik gdy będzie on już zapisany na dysku

# Liczenie similarity

In [5]:
from lib.similarity_utils import *

## Wyświetlenie danych na których pracujemy na przykładzie Julii Louis-Dreyfus

In [6]:
Julia_Louis_Dreyfus = data[data['primaryName']=="Julia Louis-Dreyfus"]

In [7]:
index = Julia_Louis_Dreyfus.index

In [8]:
data.iloc[index]

,nconst,tconst,titleType,originalTitle,isAdult,startYear,endYear,genres,category,characters,primaryName,knownForTitles
1454708,nm0000506,"[tt0462128, tt0697654, tt0697660, tt0697689, t...","[tvSeries, tvEpisode, tvEpisode, tvEpisode, tv...","[The New Adventures of Old Christine, The Bear...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1986,\N,"[Comedy, Comedy, Comedy, Comedy, Comedy, Comed...",actress,"[[""Christine Campbell""], [""Elaine Benes""], [""E...",Julia Louis-Dreyfus,"tt0462128,tt2390361,tt0098904,tt1759761"


## Definicja metody liczącej similarity

In [9]:
def prepare_pandas_row(pandas_row):
  p_list = []
  for value in pandas_row:
    try:
      p_list.append(value.tolist())
    except AttributeError:
      p_list.append(value)
  return p_list

In [36]:
def similarity(actor1, actor2) -> float:
  """metoda licząca similarity pomiędzy dwoma aktorami
  jej argumentami są dwie listy, a wartością wyjściową wartość z przedziału [-1, 1]
  metoda jest przygotowana pod dane ze zbioru JOINED_DATA"""
  if actor1[0] == actor2[0]:
    return -1
  else:
    weights = [0.6, 0.2, 0.2]
    values = [
        iou(actor1[1], actor2[1]),      # similarty ze względu na ilość wspólnych filmów
        iou(actor1[2], actor2[2]),      # similarity ze względu na rodzaj granych produkcji
        iou(actor1[7], actor2[7])       # similarity ze względu na gatunek granych produkcji
    ]
    # print(actor1[1])
    # print(actor2[1])
    no_of_columns = 3
    return sum(weights[i] * values[i] for i in range(no_of_columns)) * 2 - 1

In [37]:
def similarity_1_vs_all(data, main_actor):
  ids = []
  similarities = []
  for i in range(len(data)):
    actor = prepare_pandas_row(data.loc[i].values.tolist())
    ids.append(actor[0])
    similarities.append(similarity(main_actor, actor[0:]))
  return ids, similarities

In [38]:
def sort_two_lists(list1, list2, reverse=True):
  """sorts two lists
  reverse = True -> descending order, reverse = False -> acending order
  list2 is modified by following the sorting done on list1"""
  zipped_lists = zip(list1, list2)
  list1, list2 = zip(*sorted(zipped_lists, reverse=reverse))
  return list1, list2

## Liczenie similarity

### Dla dwóch wybranych aktorów

In [39]:
actor1 = prepare_pandas_row(data.loc[1].values.tolist())
actor2 = prepare_pandas_row(data.loc[2].values.tolist())

In [40]:
print(f"similarity dla {actor1[10]} i {actor2[10]}: {similarity(actor1, actor2)}")

similarity dla Bette Davis i Judy Garland: -0.763967363590005


In [41]:
print(f"similarity dla {actor1[10]} i {actor1[10]}: {similarity(actor1, actor1)}")

similarity dla Bette Davis i Bette Davis: -1


### Szukanie najbardziej podobnych aktorów

In [42]:
%%time
ids, values = similarity_1_vs_all(data, actor1)

CPU times: user 3min 24s, sys: 628 ms, total: 3min 25s
Wall time: 3min 26s


In [43]:
values, ids = sort_two_lists(values, ids, reverse=True)

In [44]:
print(values[:5])
print(ids[:5])

(-0.48571428571428565, -0.5017543859649123, -0.5018867924528302, -0.504, -0.5083333333333333, -0.5111111111111111, -0.5118644067796609, -0.512, -0.512, -0.512)
('nm0807252', 'nm0000254', 'nm0000569', 'nm0004133', 'nm2056274', 'nm1670494', 'nm0000047', 'nm0220016', 'nm0219099', 'nm0001735')
